# Dask Array
![img](./images/dask-array-black-text.svg)

Dask 数组协调许多 Numpy 数组，在网格内排列成块。它们支持 Numpy API 的很大一部分。

Dask 数组使用阻塞算法提供了一个并行的、大于内存的 n 维数组。简单地说：分布式 Numpy

- **并行**：使用计算机上的所有内核

- **大于内存**：通过将数组分解成许多小块，按顺序操作这些块以最大限度地减少计算的内存占用，并有效地从磁盘流式传输数据，从而让您可以处理大于可用内存的数据集.

- **阻塞算法**：通过执行许多较小的计算来执行大型计算

In [ ]:
# 创建10亿个随机数存入hdf5文件
%run prep.py -d random

In [ ]:
# 连接/创建 Dask集群

from dask.distributed import Client

client = Client(n_workers=4)
client

## 示例一

**针对一个有10亿个随机数的大数组做加法，分而治之**

In [ ]:
# 懒加载数据
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'), mode='r')
dset = f['/x']

In [ ]:
dset.shape

dask使用chunks参数将大数组切分成小块，先针对小块数组进行计算，然后将小块计算的结果再计算，最后得到结果

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(1_000_000,))
x

In [ ]:
result = x.sum()
result

In [ ]:
# 任务可视化
result.visualize()

In [ ]:
result.compute()

## 示例二

1. **构建一个 20000x20000 的正态分布随机值数组，该数组被分成 1000x1000 大小的块**
2. **沿一个轴取平均值**
3. **取每 100 个元素**

In [ ]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(20000, 20000),
                              chunks=(1000, 1000))
y = x.mean(axis=0)[::100]

In [ ]:
# 数组大小
x.nbytes / 1e9

In [ ]:
y.visualize()

In [ ]:
%%time
y.compute()